In [9]:
# ------------------------------------------------------------------------------------------
# read swmm .rpt output file, and store desired outputs
# ------------------------------------------------------------------------------------------

# setup
import pandas, os, glob, shutil, subprocess, sys
from subprocess import Popen, PIPE

# specify locations
dir_path = r'C:\Users\echelsvi\git\chelsvig_urban_pesticides\probabilistic_python'


vvwm_path = dir_path + r'\input\vvwm'
print(vvwm_path)

outfalls = r'\outfall_31_26'

Ite = 1

C:\Users\echelsvi\git\chelsvig_urban_pesticides\probabilistic_python\input\vvwm
